# Spatial-temporal Correlated Random Fields (SCRF)

Author: Cindy Chiao  
Date: 12/29/2021

The purpose of this notebook is to generate spatially and temporally correlated random fields to be used to perturb the mean prediction result of GARD downscaling method. First, we use ERA5 observation data to find the appropriate length scales of correlation both spatially and temporally. Then, we use the correlation length scales to generate random fields covering the global domain in the resolution of ERA5. The package [gstools](https://geostat-framework.readthedocs.io/projects/gstools/en/stable/#pip) is used heavily in this process. A SCRF is generated for precipitation and another for temperature (will be used to perturb both tmin and tmax prediction result). 

Ideally, the entire available observation time series of the global domain would be used in determining the correlation length scales, and the SCRF of the entire future prediction period would be generated as one contiguous dataset. However, this proves to be prohibitive in terms of computation time due to the single threaded nature of the gstools algorithm. Thus, random subsamples of the observation time series were used to find the correlation length, and the SCRF was generated in 10 year long chunks. 

To find a representative spatial correlation length, we calculate the average spatial correlation lengths of 100 20x20 degree maps of 365 day time series. The 20x20 degree maps are constrained to areas of the globe that contain major landmass, avoiding the areas where it's majority ocean. 

To find a representative temporal correlation length, we use 10,000 samples of 365 day time series, again constrained to the areas containing major landmass. 

The spatial/temporal length scales for tmin and tmax are then averaged to be the length scale used to generate SCRF for temperature. 

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import fsspec
import os
import time
import random 
import xarray as xr
import numpy as np
import pandas as pd
import gstools as gs
import rioxarray
from cmip6_downscaling.workflows.paths import make_scrf_path
from cmip6_downscaling.data.observations import get_obs

random.seed(20211228)

In [3]:
from carbonplan_trace.tiles import tiles
from carbonplan_trace.v1 import utils


## Finding correlation length scales

In [4]:
# get all 20x20 degree tiles that covers at least one 10x10 degree tile that has major land mass 

expanded_tiles = []
for tile in tiles:
    lat, lon = utils.get_lat_lon_tags_from_tile_path(tile)
    min_lat, max_lat, min_lon, max_lon = utils.parse_bounding_box_from_lat_lon_tags(lat, lon)
    
    for i in [-1, 0]:
        for j in [-1, 0]: 
            lat_tag, lon_tag = utils.get_lat_lon_tags_from_bounding_box(max_lat + (i * 10.), min_lon + (j * 10.))
            expanded_tiles.append(f'{lat_tag}_{lon_tag}')
            
expanded_tiles = list(set(expanded_tiles))
expanded_tiles = [t for t in expanded_tiles if '190W' not in t and '170E' not in t and '80N' not in t and '80S' not in t]

In [5]:
def convert_long3_to_long1(long3):
    # see https://confluence.ecmwf.int/pages/viewpage.action?pageId=149337515
    long1 = (long3 + 180) % 360 - 180
    return long1

In [6]:
variables = ['tasmax', 'tasmin', 'pr']
seasonality_period = 31

sample_length = 365
n_samples_temporal = 10000
n_tiles_spatial = 100

In [7]:
def get_spatial_length(data):
    fields = data.values
    print(np.mean(fields))
    bin_center, gamma = gs.vario_estimate(
        pos=(data.lon.values, data.lat.values),
        field=fields,
        latlon=True,
        mesh_type='structured',
    )
    spatial = gs.Gaussian(dim=2, latlon=True, rescale=gs.EARTH_RADIUS)
    spatial.fit_variogram(bin_center, gamma, sill=np.mean(np.var(fields, axis=(1, 2))))

    return spatial.len_scale

In [8]:
# spatial length scale 

for v in variables:
    print(v)
    fname = f'{v}_spatial_length_scale.csv'
    if os.path.exists(fname):
        df = pd.read_csv(fname)
        df = df.loc[df.spatial_length_scale > 1]
        print(df.spatial_length_scale.mean())
    else:
        data = get_obs(
            obs='ERA5',
            train_period_start=1980,
            train_period_end=2020,
            variables=v,
            chunking_approach=None,
        )[v]

        if v == 'pr':
            data = data * 1e6

        # go from 0-360 to -180-180 longitude 
        data['lon'] = convert_long3_to_long1(data.lon)
        data = data.reindex(lon=sorted(data.lon.values))

        # detrend 
        seasonality = (
            data.rolling({'time': seasonality_period}, center=True, min_periods=1)
            .mean()
            .groupby('time.dayofyear')
            .mean()
        )
        detrended = data.groupby("time.dayofyear") - seasonality
        detrended = detrended.transpose('time', 'lon', 'lat')
        possible_time_starts = len(detrended.time) - sample_length

        spatial_length_scale = []
        chosen_tiles = random.sample(expanded_tiles, k=n_tiles_spatial)
        for tile in chosen_tiles:
            lat, lon = utils.get_lat_lon_tags_from_tile_path(tile)
            min_lat, max_lat, min_lon, max_lon = utils.parse_bounding_box_from_lat_lon_tags(lat, lon)
            max_lat += 10
            max_lon += 10 
            t = random.randint(a=0, b=possible_time_starts)
            sub = detrended.sel(lat=slice(max_lat, min_lat), lon=slice(min_lon, max_lon)).isel(time=slice(t, t+sample_length))
            # spatial_length_scale.append(client.persist(get_spatial_length(sub), retries=1))
            l = get_spatial_length(sub)
            spatial_length_scale.append(l)
            print(tile, l)

        df = pd.DataFrame({'tile': chosen_tiles, 'spatial_length_scale': spatial_length_scale})
        df.to_csv(f'{v}_spatial_length_scale.csv')
        df = df.loc[df.spatial_length_scale > 1]
        print(df.spatial_length_scale.mean())

tasmax
437.36955028922074
tasmin
419.73912697665384
pr
404.29331338341586


In [9]:
sample_length = 100
temporal_scaler = 100.

In [10]:

# # temporal length scale 
# for v in variables:
#     print(v)
#     data = get_obs(
#         obs='ERA5',
#         train_period_start=1980,
#         train_period_end=2020,
#         variables=v,
#         chunking_approach=None,
#     )[v]

#     if v == 'pr':
#         data = data * 1e3

#     # go from 0-360 to -180-180 longitude 
#     data['lon'] = convert_long3_to_long1(data.lon)
#     data = data.reindex(lon=sorted(data.lon.values))
    
#     # detrend 
#     print('detrending')
#     seasonality = (
#         data.rolling({'time': seasonality_period}, center=True, min_periods=1)
#         .mean()
#         .groupby('time.dayofyear')
#         .mean()
#     )
#     detrended = data.groupby("time.dayofyear") - seasonality
#     detrended = detrended.transpose('time', 'lon', 'lat')
#     possible_time_starts = len(detrended.time) - sample_length
#     detrended = detrended.stack(point=['lat', 'lon'])
    
#     print('building samples')
#     points = []

#     chosen_tiles = random.choices(tiles, k=n_samples_temporal)
#     ii = random.choices(np.arange(40), k=n_samples_temporal)
#     jj = random.choices(np.arange(40), k=n_samples_temporal)

#     lat_lon_tags = [utils.get_lat_lon_tags_from_tile_path(tile) for tile in chosen_tiles]
#     bounding_boxes = [
#         utils.parse_bounding_box_from_lat_lon_tags(lat, lon)
#         for lat, lon in lat_lon_tags
#     ]

#     for bounding_box, i, j in zip(bounding_boxes, ii, jj):
#         min_lat, max_lat, min_lon, max_lon = bounding_box
#         lat = min_lat + i * 0.25
#         lon = min_lon + j * 0.25
#         points.append((lat, lon))

#     points.sort(key=lambda u: u[0])
#     sub = detrended.sel(point=points).load()
    
#     t_starts = np.array(random.choices(np.arange(possible_time_starts), k=n_samples_temporal))
#     t_ends = t_starts + sample_length

#     fields = []
#     for i, (start, end) in enumerate(zip(t_starts, t_ends)):
#         f = sub.isel(point=i, time=slice(start, end)).values
#         fields.append(f)

#     print('finding correlation length')
#     t = np.arange(sample_length) / temporal_scaler
#     bin_center, gamma = gs.vario_estimate(pos=t, field=fields, mesh_type='structured')
#     temporal = gs.Gaussian(dim=1)
#     temporal.fit_variogram(bin_center, gamma, sill=np.mean(np.var(fields, axis=1)))
    
#     print(v, temporal.len_scale * temporal_scaler)

## Generating SRCF

In [11]:
# in units of km 
spatial_scales = {
    'tasmax': 437.36955028922074,
    'tasmin': 419.73912697665384, 
    'pr': 404.29331338341586
}

# in units of day 
temporal_scales = {
    'tasmax': 3.725572253986237,
    'tasmin': 3.9400152223556937, 
    'pr': 2.0457947933307676
}

crs = 'ESRI:54008'

In [22]:
import dask 

spatial_scaler = 1.
temporal_scaler = 1.0

# @dask.delayed
def generate_scrf(
    variable,
    spatial_scales,
    temporal_scales,
    start_year,
    end_year,
    crs = 'ESRI:54008'
):
    # get the correct temporal and spatial scales to use 
    print('initializing model')
    if variable == 'temp':
        ss = np.round(np.mean([spatial_scales['tasmax'], spatial_scales['tasmin']]), 1) / spatial_scaler
        ts = np.round(np.mean([temporal_scales['tasmax'], temporal_scales['tasmin']]), 1) / temporal_scaler
        v = 'tasmax'
    elif variable == 'pr':
        ss = np.round(spatial_scales['pr'], 1) / spatial_scaler
        ts = np.round(temporal_scales['pr'], 1) / temporal_scaler 
        v = 'pr'
        
    # initializing model 
    st_dim = 2 + 1
    # # space-time anisotropy ratio given in units d / km
    # st_anis = 0.4
    model = gs.Gaussian(dim=st_dim, var=1.0, len_scale=[ss, ss, ts])  #, anis=st_anis)
    srf = gs.SRF(model, seed=2022)

    print('getting template')
    # getting the template 
    template = get_obs(
        obs='ERA5',
        train_period_start=1980,
        train_period_end=1980,
        variables=v,
        chunking_approach=None,
    )[v].isel(time=0)
    projected = template.rio.write_crs('EPSG:4326').rio.reproject(crs, shape=(360, 720))
    # transform from m to km 
    
    x = np.round(projected.x.values / (1000 * spatial_scaler), 2) 
    y = np.round(projected.y.values / (1000 * spatial_scaler), 2)
    
    # x = projected.x / (1000 * spatial_scaler)
    # xmin = x.min()
    # y = projected.y / (1000 * spatial_scaler)
    # ymin = y.min()
    # x = x - xmin + 1
    # y = y - ymin + 1
    
    # get number of days in the period 
    t = pd.date_range(
        start=f'{start_year}-01-01', 
        end=f'{end_year}-12-31', 
        freq='D')

    print('generating field')
    # generate field 
    time = np.arange(len(t)) / temporal_scaler
    field = srf.structured((x, y, time))

    print('making dataarray')
    field = xr.DataArray(
        field,
        dims=['x', 'y', 'time'],
        coords=[projected.x, projected.y, t],
    ).rio.write_crs(crs)

    # reprojecting 
    print('reprojecting')
    field = field.transpose('time', 'y', 'x').rio.reproject('EPSG:4326')
    field = field.rename({'x': 'lon', 'y': 'lat'})
    
    print('saving')
    # save to file     
    if variable == 'temp':
        for v in ['tasmax', 'tasmin']:
            path = make_scrf_path('ERA5', v, start_year, end_year)
            store = fsspec.get_mapper('az://flow-outputs/intermediate/' + path)
            field.to_dataset(name='scrf').to_zarr(store, mode="w", consolidated=True)
    else:
        path = make_scrf_path('ERA5', 'pr', start_year, end_year)
        store = fsspec.get_mapper(path)
        field.to_dataset(name='scrf').to_zarr(store, mode="w", consolidated=True)                       
    
    return ("processed", path)

In [ ]:
# %%debug --breakpoint /srv/conda/envs/notebook/lib/python3.9/site-packages/gstools/covmodel/base.py:533

variable = 'pr'
start_year = 2011
end_year = 2020

generate_scrf(
    variable=variable,
    spatial_scales=spatial_scales,
    temporal_scales=temporal_scales,
    start_year=start_year,
    end_year=end_year,
)

initializing model
getting template
generating field


In [16]:
variable = 'temp'
start_year = '2011'
end_year = '2015'

# get the correct temporal and spatial scales to use 
print('initializing model')
if variable == 'temp':
    ss = np.round(np.mean([spatial_scales['tasmax'], spatial_scales['tasmin']]), 1) / spatial_scaler
    ts = np.round(np.mean([temporal_scales['tasmax'], temporal_scales['tasmin']]), 1) / temporal_scaler
    v = 'tasmax'
elif variable == 'pr':
    ss = np.round(spatial_scales['pr'], 1) / spatial_scaler
    ts = np.round(temporal_scales['pr'], 1) / temporal_scaler 
    v = 'pr'

print('getting template')
# getting the template 
template = get_obs(
    obs='ERA5',
    train_period_start=1980,
    train_period_end=1980,
    variables=v,
    chunking_approach=None,
)[v].isel(time=0)

initializing model
getting template


In [19]:
template.shape

(721, 1440)

<xarray.DataArray 'tasmax' (y: 360, x: 720)>
array([[240.    , 238.1875, 237.5   , ..., 255.    , 254.875 , 254.375 ],
       [252.375 , 252.5   , 252.625 , ..., 253.5625, 253.25  , 252.625 ],
       [254.    , 254.75  , 255.    , ..., 245.5   , 244.6875, 243.3125],
       ...,
       [241.875 , 241.5   , 241.375 , ..., 262.    , 261.4375, 260.875 ],
       [247.3125, 247.125 , 247.    , ..., 259.625 , 256.625 , 253.8125],
       [260.0625, 260.    , 260.    , ..., 246.25  , 246.3125, 246.4375]],
      dtype=float32)
Coordinates:
  * x            (x) float64 -2.001e+07 -1.995e+07 ... 1.993e+07 1.999e+07
  * y            (y) float64 8.984e+06 8.934e+06 ... -8.923e+06 -8.973e+06
    time         datetime64[ns] 1980-01-01
    spatial_ref  int64 0
Attributes:
    long_name:       10 metre U wind component
    nameCDM:         10_metre_U_wind_component_surface
    nameECMWF:       10 metre U wind component
    product_type:    analysis
    shortNameECMWF:  10u
    standard_name:   eastward_wind
    units:           m s**-1

In [ ]:
projected = template.rio.write_crs('EPSG:4326').rio.reproject(crs)
# transform from m to km 

x = np.round(projected.x.values / (1000 * spatial_scaler), 2) 
y = np.round(projected.y.values / (1000 * spatial_scaler), 2)

# get number of days in the period 
t = pd.date_range(
    start=f'{start_year}-01-01', 
    end=f'{end_year}-12-31', 
    freq='D')

In [14]:
# ss = 429
# ts = 3.8

# model = gs.Gaussian(dim=3, var=1.0, len_scale=[ss, ss, ts / temporal_scaler])
# srf = gs.SRF(model, seed=0)

# step = 25
# nx = len(x)
# ny = len(y)
# nt = len(t)

# xx = np.arange(0, nx*step, step)
# yy = np.arange(0, ny*step, step)
# tt = np.arange(0, nt) / temporal_scaler

# t1 = time.time()
# field = srf.structured((xx, yy, tt))
# t2 = time.time()

# print((t2-t1)/60)

In [15]:
# len(x), len(y), len(t)

In [16]:
# field = srf.structured((x, y, np.arange(len(t)) / temporal_scaler))

In [17]:
len(x)

1469

In [18]:
len(y)

660

In [19]:
len(t)

1826

In [20]:
ss

428.6

In [21]:
ts

3.8

In [22]:
# things to tweak 
# num lat, num lon, num time 
# ts and ss -- ratio between the two and how they relate to the scale of lat/lon/time 

# all my set up & 10 timepoints pass 
# all my set up & 100 timepoints crashed 
# all their set up & 100 time points? (switched into Gaussian, and len x and len y are my setup) -- this passed 
# everything as above but switch to my length scales -- this passed
# everything as above but exported and round x and y that are mine setup -- this passed too 

# all their set up & 5 years -- this crashed 

In [ ]:
# fix the seed for reproducibility
seed = 20170521
# 1st spatial axis of 50km with a resolution of 1km
x = np.arange(0, 1469, 1.0)
# 2nd spatial axis of 40km with a resolution of 1km
y = np.arange(0, 660, 1.0)

# x = np.round(x.values, 2)
# y = np.round(y.values, 2)

# half daily timesteps over three months
t = np.arange(0.0, 365*3, 1.0)

# total spatio-temporal dimension
st_dim = 2 + 1
# space-time anisotropy ratio given in units d / km
st_anis = 0.4

# an exponential variogram with a corr. lengths of 5km, 5km, and 2d
model = gs.Gaussian(dim=st_dim, var=1.0, len_scale=5.0, anis=st_anis)
# model = gs.Gaussian(dim=3, var=1.0, len_scale=[ss, ss, ts])
srf = gs.SRF(model, seed=seed)

pos, time = [x, y], [t]

# the Gaussian random field
field = srf.structured(pos + time)

In [ ]:
print('done')

In [ ]:
field

In [ ]:
# 6 years and above generated a fortran failure 
# 5 years simply crashes the notebook without any error (no trace back or error log, the notebook becomes unresponsive and generate blanks when a cell is ran)
# try one year? also crashes 
# try precip? 1 year of precip worked 
# try precip 5 year -- this crashed 
# try other x/y/t configuration -- for 5 year this also crashed (temporal scale = 100, x and y starts with 0 with step of 25)

In [17]:
import dask
from dask.distributed import Client
from dask_gateway import Gateway

client = Client(n_workers=2, threads_per_worker=2, resources={"workertoken": 1})
client

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes
/srv/conda/envs/notebook/lib/python3.9/contextlib.py:126: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 4,Total memory: 251.72 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39793,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 251.72 GiB
Comm: tcp://127.0.0.1:35641,Total threads: 2
Dashboard: http://127.0.0.1:40159/status,Memory: 125.86 GiB
Nanny: tcp://127.0.0.1:37029,


In [18]:
tasks = []

for variable in ['temp', 'pr']:
    for start_year in np.arange(2011, 2100, 5):
        end_year = str(start_year + 4)
        start_year = str(start_year)
        print(variable, start_year, end_year)
        
        tasks.append(
            client.compute(
                generate_scrf(
                    variable=variable,
                    spatial_scales=spatial_scales,
                    temporal_scales=temporal_scales,
                    start_year=start_year,
                    end_year=end_year,
                ),
                resources={"workertoken": 1},
            )
        )

temp 2011 2015
temp 2016 2020
temp 2021 2025
temp 2026 2030
temp 2031 2035
temp 2036 2040
temp 2041 2045
temp 2046 2050
temp 2051 2055
temp 2056 2060
temp 2061 2065
temp 2066 2070
temp 2071 2075
temp 2076 2080
temp 2081 2085
temp 2086 2090
temp 2091 2095
temp 2096 2100
pr 2011 2015
pr 2016 2020
pr 2021 2025
pr 2026 2030
pr 2031 2035
pr 2036 2040
pr 2041 2045
pr 2046 2050
pr 2051 2055
pr 2056 2060
pr 2061 2065
pr 2066 2070
pr 2071 2075
pr 2076 2080
pr 2081 2085
pr 2086 2090
pr 2091 2095
pr 2096 2100


In [19]:
results = dask.compute(tasks, retries=1, resources={"workertoken": 1})[0]
results

[<Future: pending, key: generate_scrf-84df808a-e46b-4b06-b713-cff7a2bdbaea>,
 <Future: pending, key: generate_scrf-4cbc8224-ce17-4bc0-b5f7-b5b8bc012889>,
 <Future: pending, key: generate_scrf-4e5b973f-681f-4958-a262-b24d1c13096a>,
 <Future: pending, key: generate_scrf-e8ef5bdb-c315-4538-b3b9-86caba158594>,
 <Future: pending, key: generate_scrf-f084cd37-bb19-4504-83df-c29010ebc083>,
 <Future: pending, key: generate_scrf-da90536f-fba5-4fb2-b10a-e30dd7d23791>,
 <Future: pending, key: generate_scrf-d1128d63-9dcd-4a79-a561-f44dbd5fb2e2>,
 <Future: pending, key: generate_scrf-0888c8dd-578f-46ad-a620-e09f5ca51b06>,
 <Future: pending, key: generate_scrf-fbc434ab-e0df-4e1b-971c-5ad25ed9cac8>,
 <Future: pending, key: generate_scrf-bab8377b-47d1-4ef2-ba79-d5c79168985c>,
 <Future: pending, key: generate_scrf-64a15f32-5479-4571-b367-b9a73a149f26>,
 <Future: pending, key: generate_scrf-cc916123-1d86-4178-a619-b3c0adac894d>,
 <Future: pending, key: generate_scrf-7f3be31b-e1ca-44cd-8319-185bfe1be3e8>,

In [ ]:
results

In [ ]:
template = get_obs(
    obs='ERA5',
    train_period_start=1980,
    train_period_end=1980,
    variables=v,
    chunking_approach=None,
)[v].isel(time=0)
projected = template.rio.write_crs('EPSG:4326').rio.reproject(crs)
x = projected.x
y = projected.y

In [17]:
x

<xarray.DataArray 'x' (x: 1469)>
array([-20023848.174586, -19996582.1934  , -19969316.212214, ...,
        19948080.243954,  19975346.22514 ,  20002612.206326])
Coordinates:
  * x            (x) float64 -2.002e+07 -2e+07 -1.997e+07 ... 1.998e+07 2e+07
    time         datetime64[ns] 1980-01-01
    spatial_ref  int64 0
Attributes:
    axis:           X
    long_name:      x coordinate of projection
    standard_name:  projection_x_coordinate
    units:          metre

In [18]:
y

<xarray.DataArray 'y' (y: 660)>
array([ 8995723.298596,  8968438.743853,  8941154.18911 , ..., -8930229.167275,
       -8957513.722018, -8984798.276761])
Coordinates:
  * y            (y) float64 8.996e+06 8.968e+06 ... -8.958e+06 -8.985e+06
    time         datetime64[ns] 1980-01-01
    spatial_ref  int64 0
Attributes:
    axis:           Y
    long_name:      y coordinate of projection
    standard_name:  projection_y_coordinate
    units:          metre

In [4]:
step = 25
nx = 100
ny = 100
nt = 365*30+8

x = np.arange(0, nx*step, step)
y = np.arange(0, ny*step, step)
t = np.arange(0, nt)

t1 = time.time()
field = srf.structured((x, y, t))
t2 = time.time()

print((t2-t1)/60)

62.33059689203898


In [5]:
step = 25
nx = 100
ny = 100
nt = 365*30+8

x = np.arange(0, nx*step, step)
y = np.arange(0, ny*step, step)
t = np.arange(0, nt)

len(x) * len(y) * len(t)

109580000

In [4]:
# 10 year chunks 
# 

step = 25
nx = 360*4
ny = 180*4
nt = 365

x = np.arange(0, nx*step, step)
y = np.arange(0, ny*step, step)
t = np.arange(0, nt)

t1 = time.time()
field = srf.structured((x, y, t))
t2 = time.time()

print((t2-t1)/60)

217.89220894575118


In [18]:
import matplotlib.pyplot as plt
import random

In [ ]:
for _ in np.arange(30):
    i = random.randint(0, 365)
    plt.imshow(field[:, :, _])
    plt.show()
    plt.close()
    
# import numpy as np
# import cv2
# size = 720*16//9, 720
# duration = 2
# fps = 25
# out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]), False)
# for _ in range(fps * duration):
#     data = np.random.randint(0, 256, size, dtype='uint8')
#     out.write(data)
# out.release()

In [6]:
step = 25
nx = 360*4
ny = 180*4
nt = 365

x = np.arange(0, nx*step, step)
y = np.arange(0, ny*step, step)
t = np.arange(0, nt)

len(x) * len(y) * len(t)

378432000

In [7]:
378432000 / 109580000

3.453476911845227

In [8]:
217.89220894575118 / 62.33059689203898

3.4957503988475507

In [9]:
217.89220894575118 * 30 / 60

108.94610447287559

In [4]:
ds = xr.DataArray(
    np.random.rand(360*4, 180*4, 365*30+8),
    dims=['lon', 'lat', 'time'],
    coords=[np.arange(360*4), np.arange(180*4), np.arange(365*30+8)]
)

In [6]:
ds.nbytes / 1e9

90.8900352